# A look into creating interactive maps with folium

We'll be using the individual datasets at first, to get individual countries' maps. Once we have the work on individual countries, we'll start trying to build a european map

#We start by importingimport countriesAndCities the required packages

In [ ]:
import countriesAndCities
import folium
import geopandas as gpd
import pandas as pd
from shapely import wkt

# 1. Austria

We build the dataset the same way we built it for the unified database

In [ ]:
countries = {}

In [ ]:
countries['Austria'] = countriesAndCities.listCities('Austria')

In [ ]:
countries['Austria'][0] = 'Wien'

# 1.1 Stations

In [ ]:
stationsAustria = 'Austria/GIP_Betriebsstellen_DelEUV_JSON.json'
stationsAustriaFrame = gpd.read_file(stationsAustria)

In [ ]:
columnsToRemove = ['BSTS_ID', 'DB640_CODE', 'OBJECTID', 'GIP_OBID', 'EXTERNALID', 'REGIONALCO', 'VALIDFROM', 'VALIDTO', 'OWNER_NAME', 'PV_EVA_NR', 'ANZ_AUFZUG', 'ANZ_FAHRTR', 'ANZ_UHREN',
                  'ANZ_AKUSTI','ANZ_OPTISC', 'INFOPOINT', 'MUEZ', 'MUEZ_KURZ', 'HILFE_MOBI', 'ANZ_ROLLST', 'ANZ_E_LADE', 'RUD_PARKPL', 'VERIFIZIER',
                  'PUBL_WLAN', 'MUEZ_LANG', 'BEMERKUNG']

In [ ]:
for column in columnsToRemove:
    stationsAustriaFrame = stationsAustriaFrame.drop(column, axis=1)

In [ ]:
dfStationsAustria = []
for city in countries['Austria']:
    tempFrame = stationsAustriaFrame.loc[stationsAustriaFrame['NAME_FPL'].str.contains(city + ' ')]
    dfStationsAustria.append(tempFrame)

In [ ]:
workFrameAustria = gpd.GeoDataFrame(pd.concat(dfStationsAustria), crs = 31287)

In [ ]:
workFrameAustria = gpd.GeoDataFrame(pd.concat(dfStationsAustria), crs = 31287)

In [ ]:
workFrameAustria

In [ ]:
workFrameAustria['NAME_FPL'].values[0]

# 1.2 Lines

In [ ]:
linesAustria = 'Austria/GIP_Strecken_MLA.json'
linesAustriaFrame = gpd.read_file(linesAustria)

In [ ]:
uselessColumns = ['GIP_OBID', 'BST_ID', 'FOW_NAME', 'FRC_NAME', 'REGION', 'VALIDFROM', 'VALIDTO', 'CROSSSECT', 'CROSS_NAME', 
                  'ELEKTRI', 'EXPDATE']

for column in uselessColumns:
    linesAustriaFrame = linesAustriaFrame.drop(column, axis=1)

In the case of the lines, we can't remove any rows of the dataframe. Every line has a bit of information, that cannot be replicated based on other rows

# 2. Visualisation

We now have two sets of working data. We can visualise this data on interactive maps using folium

In [ ]:
trainMap = folium.Map(location = [47.5, 14.6], tiles = 'OpenStreetMap', zoom_start=12)



We can now add markers, based on the different train stations, onto the map

In [ ]:
austriaStations = [[point.xy[1][0], point.xy[0][0]] for point in workFrameAustria.geometry ]

In [ ]:
for i in range (len(austriaStations)):
    coordinates = austriaStations[i]
    trainMap.add_child(folium.Marker(location = coordinates, popup = workFrameAustria['NAME_FPL'].values[i]))
    

In [ ]:
trainMap

In [ ]:
for line in linesAustriaFrame.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

# 2. Germany

We can do the exact same for the German train lines

In [ ]:
countries['Germany'] = countriesAndCities.listCities('Germany')
countries['Germany'][2] = 'München'
countries['Germany'][4] = 'Frankfurt'

# 2.1 German Stations

In [ ]:
stations = 'Germany/railwayStationNodes.geojson'

deutscheBahnStations = gpd.read_file(stations)

In [ ]:
deutscheBahnStations = deutscheBahnStations.drop('formOfNode', axis = 1)
deutscheBahnStations = deutscheBahnStations.drop('id', axis = 1)

In [ ]:
dfListStations = []
for city in countries['Germany']:
    tempFrame = deutscheBahnStations.loc[deutscheBahnStations['geographicalName'].str.contains(city + ' ')]
    dfListStations.append(tempFrame)

In [ ]:
workFrameStationsGermany = gpd.GeoDataFrame(pd.concat(dfListStations, ignore_index=True), crs=4258)

In [ ]:
workFrameStationsGermany

# 2.2 German train lines

In [ ]:
deutscheBahnLines = gpd.read_file('Germany/railwayLines.geojson')

In [ ]:
dfListLines = []
for city in countries['Germany']:
    tempFrameLines = deutscheBahnLines.loc[deutscheBahnLines['geographicalName'].str.contains(city + ' ')]
    dfListLines.append(tempFrameLines)

In [ ]:
workFrameLines = gpd.GeoDataFrame(pd.concat(dfListLines, ignore_index = True), crs = 4258)

In [ ]:
workFrameLines = workFrameLines.drop_duplicates(subset='railwayLineCode')

# 2.3 Visualising German data

In [ ]:
germanStationsList = [[point.xy[1][0], point.xy[0][0]] for point in workFrameStationsGermany.geometry ]

In [ ]:
for i in range(len(germanStationsList)):
    coordinates = germanStationsList[i]
    trainMap.add_child(folium.Marker(location = coordinates, popup= workFrameStationsGermany['geographicalName'].values[i]))

In [ ]:
trainMap

In [ ]:
for line in deutscheBahnLines.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap

The German data is awfully incomplete. We can add further information on the German rail network by using a global database

In [ ]:
globalData = 'wld_trs_railways_wfp.csv'
worldData = pd.read_csv(globalData)

In [ ]:
Germany = worldData[worldData['country'] == 'Germany']

In [ ]:
Germany['shape'] = Germany['shape'].apply(wkt.loads)

In [ ]:
geodata = gpd.GeoSeries(Germany['shape'])

In [ ]:
Germany.set_geometry('shape')

In [ ]:
for line in geodata.geometry:
    lineTuple = []
    lineTupleInvert = list(line[0].coords)
    for i in range (len(lineTupleInvert)):
        lineTuple.append([lineTupleInvert[i][1], lineTupleInvert[i][0]])
    folium.PolyLine(lineTuple).add_to(trainMap)

In [ ]:
trainMap